In [1]:
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import pandas as pd
from tqdm import tqdm 

In [2]:
# koELECTRA 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

In [3]:
''' 모델가중치를 불러오거나 체크포인트를 불러올 때 실행 '''

# 모델의 상태 딕셔너리를 로드합니다.
model_state_dict = torch.load("Ternary_model_state_dict_epoch_15.pt")

# 모델을 생성하고 상태를 로드합니다.
model = ElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=3)
model.load_state_dict(model_state_dict)

# 옵티마이저의 상태 딕셔너리를 로드합니다.
optimizer_state_dict = torch.load("Ternary_optimizer_state_dict_epoch_15.pt")

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at koelectra-base-v3-discriminator and are newly initialized: ['classifier.out_p

In [4]:
# 변경하고자 하는 Dropout 비율
new_dropout_rate = 0.2

# 모든 Dropout 레이어의 비율 변경
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Dropout):
        module.p = new_dropout_rate


In [ ]:
# 모델 구조 확인
print(model)

In [ ]:
# 장치 설정 (GPU 사용을 위해)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 실제로 예측해보기

In [6]:
# 한국어 문장을 입력으로 받아서 예측 라벨을 출력하는 함수
def predict_label(sentence, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
        return predicted_label

In [7]:
df = pd.read_csv("../dataset/2019산림복지_본문_맞춤법검사.txt", encoding='UTF-8')
df

,Unnamed: 0,split_str,org_idx,correct_str
0,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...
1,1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤 원해 평생학습원 이세인...
2,2,안녕하세요 메리썽입니다,1,안녕하세요. 메리 성입니다
3,3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1,혁신도시에 대해서는 많이들 들어보셨을 거라고 생각이 드는데요
4,4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1,현재 혁신도시 시즌2가 지속해 진행되고 있는 만큼 지금 시기에 이에 대해 이야기해보...
...,...,...,...,...
164722,164722,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126,의 건립비로 지출하는 기부금 9 중소기업협동조합법에 따른 중소기업중앙회에 중소기업의...
164723,164723,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126,이하 이 호에서 같다가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경기...
164724,164724,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...
164725,164725,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126,한국국제학교 22 천진 한국국제학교 23 테헤란 한국학교 ㅇ 법정기부금 단체 중 전...


In [8]:
# 한국어 문장 입력 받기
korean_sentences = df['correct_str'].tolist()

# 예측 라벨 출력
emotion_labels = {0:'부정', 1:'중립', 2:'긍정'}
predicted_label = [emotion_labels[predict_label(korean_sentence, model, tokenizer, device)] for korean_sentence in tqdm(korean_sentences)]
df['label'] = predicted_label

 96%|█████████▌| 158275/164727 [1:44:56<03:49, 28.11it/s]  

In [ ]:
ddff = df[['correct_str','label','org_idx']]
ddff

,correct_str,label
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,긍정
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤 원해 평생학습원 이세인...,긍정
2,안녕하세요. 메리 성입니다,긍정
3,혁신도시에 대해서는 많이들 들어보셨을 거라고 생각이 드는데요,부정
4,현재 혁신도시 시즌2가 지속해 진행되고 있는 만큼 지금 시기에 이에 대해 이야기해보...,긍정
...,...,...
164722,의 건립비로 지출하는 기부금 9 중소기업협동조합법에 따른 중소기업중앙회에 중소기업의...,긍정
164723,이하 이 호에서 같다가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경기...,부정
164724,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,긍정
164725,한국국제학교 22 천진 한국국제학교 23 테헤란 한국학교 ㅇ 법정기부금 단체 중 전...,긍정


In [15]:
ddff['label'].value_counts()

긍정    116803
부정     37528
중립     10396
Name: label, dtype: int64

In [16]:
ddff[ddff['label'] == '중립']

,correct_str,label
5,여기서 중요한 키워드는 공공기관 지방 이전과 미래형 도시가 되겠는데요,중립
8,경제 문화 산업 인구 등등 수도권의 영향을 받지 않는 것들이 없지요.,중립
9,공공기관 또한 예외가 아닐 수 없어요.,중립
10,수도권 과밀 해소와 자립형 지방화를 이룰 수 있는 것이 바로 공공기관 지방 이전이 ...,중립
11,이를 위해서는 적극적인 수도권 발전정책과 지방육성 정책이 필요한데요. 공공기관 지방...,중립
...,...,...
164556,이에 따라 중앙행정기관이나 지자체가 해양 이용 개발 계획을 세우거나 지구 구역을 지...,중립
164558,이에 따라 태양광 발전시설을 목적으로 산지를 사용한 뒤에는 나무를 심고 원상태로 복...,중립
164573,2019년 한 해 잘 시작하셨나요.,중립
164668,다만 국 공유재산의 평가는 사업 승인이 있는 날의 다음 날을 기준으로 평가하되 용도...,중립


In [ ]:
ddff.rename(columns={'correct_str': 'sentence'}, inplace=True)
ddff.rename(columns={'label': 'predicted_sentiment'}, inplace=True)
ddff.to_csv("./Ternary_classification_2019.csv", index=False, encoding='UTF-8')

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
